In [1]:
%conda install langchain-text-splitters langchain-community langgraph
%conda install python-dotenv
%conda install langchain[mistralai]
%conda install langchain-community
%conda install langchain-qdrant

Channels:
 - huggingface
 - pypi
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Channels:
 - huggingface
 - pypi
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Channels:
 - huggingface
 - pypi
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Channels:
 - huggingface
 - pypi
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
Channels:
 - huggingface
 - pypi
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packag

In [2]:
%pip install langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
os.getenv("MISTRAL_API_KEY")

'68A35kmZcxaL7ICU8s6SqnkfzbcDT9dG'

In [4]:
import getpass
import os

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("mistral-large-latest", model_provider="mistralai")

In [5]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="Llama3.1")

In [6]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

client = QdrantClient()

# Create the collection

collection_name = "test1"

if not client.collection_exists(collection_name=collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
    )

vector_store = QdrantVectorStore(
    client=client,
    collection_name=collection_name,
    embedding=embeddings,
)

In [7]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)


In [9]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


In [10]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [12]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

Task Decomposition is a process where a complex task is broken down into simpler, more manageable subtasks. This is often done to make the task easier to understand, execute, and optimize. In the context of AI systems like HuggingGPT, task decomposition involves parsing user requests into multiple tasks with defined attributes, which are then distributed to expert models for execution.
